In [1]:
# install the libraries needed for the project
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1

In [2]:
pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00


In [3]:
# a class to create a question answering system based on information retrieval
from langchain.chains import RetrievalQA
# a class to create text embeddings using HuggingFace templates
from langchain.embeddings import HuggingFaceEmbeddings
# a class to create a neural language model using LlamaCpp, a C++ implementation of GPT-3
from langchain.llms import LlamaCpp
# a class for splitting text into fixed-sized chunks with an optional overlay
from langchain.text_splitter import RecursiveCharacterTextSplitter
# a class to create a vector index using FAISS, a library for approximate nearest neighbor search
from langchain.vectorstores import FAISS
# a class for loading PDF documents from a directory
from langchain.document_loaders import PyPDFDirectoryLoader

In [4]:
# load PDF files from a directory
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/support-docs")
data = loader.load()
# print the loaded data, which is a list of tuples (file name, text extracted from the PDF)
print(data)

[Document(page_content='v1.0 | January 2024 | TN-2024\nTECH NOTE\nBacking Up and Restoring\nMicrosoft SQL Server\nDatabases Using Nutanix\nVolumes and Transact-SQL\nSnapshots', metadata={'source': '/content/drive/MyDrive/support-docs/TN-2024-Back-Up-Restore-SQL-Server-Volumes-T-SQL-Snapshots.pdf', 'page': 0}), Document(page_content='Legal\n© 2024 Nutanix, Inc. All rights reserved. Nutanix, the Enterprise Cloud Platform, the\nNutanix logo and the other Nutanix products, features, and/or programs mentioned\nherein are registered trademarks or trademarks of Nutanix, Inc. in the United States\nand other countries. All other brand and product names mentioned herein are for\nidentification purposes only and are the property of their respective holder(s), and\nNutanix may not be associated with, or sponsored or endorsed by such holder(s). This\ndocument is provided for informational purposes only and is presented "as is" with no\nwarranties of any kind, whether implied, statutory or otherwise

In [5]:
# split the extracted data into text chunks using the text_splitter, which splits the text based on the specified number of characters and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)
# print the number of chunks obtained
len(text_chunks)

47

In [6]:
# download the embeddings to use to represent text chunks in a vector space, using the pre-trained model "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# create embeddings for each text chunk using the FAISS class, which creates a vector index using FAISS and allows efficient searches between vectors
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [9]:
# Import the neural language model using the LlamaCpp class, which allows you to use a GPT-3 model in C++ with various parameters such as temperature, top_p, verbose and n_ctx (maximum number of tokens that can be generated)
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q3_K_S.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

In [10]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [11]:
query = "what is nutanix solution for Microsoft sql server database backup and recovery"

In [12]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: can you tell me about the mssql databases?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    4123.00 ms
llama_print_timings:      sample time =      71.31 ms /   102 runs   (    0.70 ms per token,  1430.39 tokens per second)
llama_print_timings: prompt eval time =  144748.62 ms /   252 tokens (  574.40 ms per token,     1.74 tokens per second)
llama_print_timings:        eval time =   73173.98 ms /   101 runs   (  724.49 ms per token,     1.38 tokens per second)
llama_print_timings:       total time =  218110.68 ms /   353 tokens


Answer:  Yes, I can help you with that. MSSQL stands for Microsoft SQL Server. It is a relational database management system (RDBMS) that allows you to store and manage large amounts of data in a structured way. MSSQL is widely used for storing and managing data in various industries such as finance, healthcare, retail, and more. It offers features such as high availability, scalability, security, and performance, making it a popular choice for mission-critical applications.
Input Prompt: how can mssql database protected by using nutanix 


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4123.00 ms
llama_print_timings:      sample time =      44.51 ms /    70 runs   (    0.64 ms per token,  1572.68 tokens per second)
llama_print_timings: prompt eval time =  232607.59 ms /   382 tokens (  608.92 ms per token,     1.64 tokens per second)
llama_print_timings:        eval time =   48952.11 ms /    69 runs   (  709.45 ms per token,     1.41 tokens per second)
llama_print_timings:       total time =  281712.37 ms /   451 tokens


Answer:  MSSQL database protection with Nutanix involves configuring Nutanix Kubernetes Engine and SQL Server, utilizing Transact-SQL and Nutanix Volumes solution for containerized SQL Server databases, automating data-loading tasks with HammerDB, and using T-SQL Snapshot Backup and Restore for Kubernetes.
Input Prompt: is transaction log backup available as part of nutanix backup and recovery , how it works?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4123.00 ms
llama_print_timings:      sample time =     102.15 ms /   147 runs   (    0.69 ms per token,  1439.06 tokens per second)
llama_print_timings: prompt eval time =  179038.60 ms /   309 tokens (  579.41 ms per token,     1.73 tokens per second)
llama_print_timings:        eval time =  104630.13 ms /   146 runs   (  716.64 ms per token,     1.40 tokens per second)
llama_print_timings:       total time =  283938.83 ms /   455 tokens


Answer: 
Transaction Log Backup in Microsoft SQL Server databases provides a way to capture the changes made to the database since the last full or differential backup, enabling quick and easy restoration of previous versions of the database. Nutanix Volumes snapshot-based approach can provide transaction log backups by taking snapshots of the transaction logs at regular intervals. The frequency of these snapshots depends on the specific needs of your application and storage infrastructure. To restore a transaction log backup, you can use the Transact-SQL RESTORE LOG command to apply the changes made to the database since the last full or differential backup. It's important to note that transaction logs are only available for databases that have been configured to use them.
Input Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
